# geomPie()

A pie chart is a circular statistical graphic, which is divided into slices to illustrate numerical proportion.

1. [Default presentation](#1.-Default-presentation)

    1.1. [Basic pie chart](#1.1.-Basic-pie-chart)
    
    1.2. [Improve appearance](#1.2.-Improve-appearance)   
    
    1.3. [Adding labels to pie sectors](#1.3.-Adding-labels-to-pie-sectors) 
    
    1.4. [Use "count2d" statistical transformation](#1.4.-Use-"count2d"-statistical-transformation)
      
        
2. [Pie size depending on data](#2.-Pie-size-depending-on-data)

      
3. [Explode](#3.-Explode)

In [1]:
%useLatestDescriptors
%use lets-plot
%use dataframe

In [2]:
LetsPlot.getInfo()

Lets-Plot Kotlin API v.4.4.1. Frontend: Notebook with dynamically loaded JS. Lets-Plot JS v.3.2.0.

In [3]:
val blankTheme = theme(line=elementBlank(), axis=elementBlank())

In [4]:
val (w, h) = 400 to 250

val data = mapOf(
    "name" to listOf('a', 'b', 'c', 'd', 'b'),
    "value" to listOf(40, 90, 10,  50,  20)
)

val p = letsPlot(data) + ggsize(w,h)

## 1. Default presentation

### 1.1. Basic pie chart

Use "identity" statistical transformation to leave the data unchanged.

In [5]:
p + geomPie(stat = Stat.identity) { slice = "value"; fill = "name" }

### 1.2. Improve appearance

- add stroke (`stroke` and `strokeColor`)
- make the pie bigger (`size`)
- add hole to draw donut-like chart (`hole`)
- use blank theme (remove axis and grid)
- use better colors

In [6]:
p +
    geomPie(stat = Stat.identity,
            size = 20, stroke = 1, strokeColor = "white", hole = 0.5) { slice = "value"; fill = "name" } +
    blankTheme + 
    scaleFillBrewer(palette = "Set1")

### 1.3. Adding labels to pie sectors

Let's label the sectors with their names - configure annotations via `layerLabels()` function: 

In [7]:
p + 
    geomPie(
        stat = Stat.identity,
        size = 20, stroke = 1, strokeColor = "white", hole = 0.5,
        labels = layerLabels().line("@name").size(16)
    ) { slice = "value"; fill = "name" } +
    blankTheme + 
    theme().legendPositionNone() +
    scaleFillBrewer(palette = "Set1")

### 1.4. Use "count2d" statistical transformation

`geomPie()` uses `count2d` stat by default. 
It allows to make a slice sizes proportional to the number of cases in each group  (or if the weight aesthetic is supplied, the sum of the weights). Also `count2d` provides variables for proportion ('..prop..') and proportion in percent ('..proppct..'). 

Using `layerTooltips()` prepare the information for tooltips by adding the variables provided by 'count2d':

In [8]:
val tooltipContent = layerTooltips()
                        .line("count|@{..count..} (@{..prop..})")
                        .line("total|@{..sum..}")
                        .format("..prop..", ".0%")
                        .format("..count..", ".1f")
                        .format("..sum..", ".1f")

Apply 'count2d' to get slices proportional to the number of cases.

In [9]:
p +
    geomPie(
        size = 20, stroke = 1, strokeColor = "white", hole = 0.5,
        labels = layerLabels().line("@name").size(16),
        tooltips = tooltipContent
    ) { fill = "name" } +
    blankTheme + 
    theme().legendPositionNone() +
    scaleFillBrewer(palette = "Set1")

Compute weighted sum instead of simple count with aesthetic `weight`.

In [10]:
p +
    geomPie(
        size = 20, stroke = 1, strokeColor = "white", hole = 0.5,
        labels = layerLabels().line("@name").size(16),
        tooltips = tooltipContent
    ) { fill = "name"; weight = "value" } +
    blankTheme + 
    theme().legendPositionNone() +
    scaleFillBrewer(palette = "Set1")    

Order sectors by count.

The following ordering rule is used for the pie chart: the first slice goes to the left of 12 o'clock and others go clockwise.

In [11]:
p +
    geomPie(
        size = 20, stroke = 1, strokeColor = "white", hole = 0.5,
        labels = layerLabels().line("@name").size(16),
        tooltips = tooltipContent
    ) { fill = asDiscrete("name", orderBy = "..count.."); weight = "value" } +
    blankTheme + 
    theme().legendPositionNone() +
    scaleFillBrewer(palette = "Set1")

## 2. Pie size depending on data

Make the size of the pie chart dependent on the data: map total count ('..sum..' variable) to the `size`.

Note that it has its own special representation of the size in the legend.

In [12]:
val data2 = mapOf(
    "x" to listOf(1, 1, 1, 1, 1, 1.5, 1.5,   2,   2,   2),
    "y" to listOf(1, 1, 1, 1, 1,   2,   2, 1.5, 1.5, 1.5),
    "s" to listOf(3, 1, 2, 1, 4,   1,   3,   3,   3,  1),
    "n" to listOf('a', 'b', 'a', 'c', 'a',  'a', 'b', 'c', 'a',  'b')
)


letsPlot(data2) +
    geomPie(hole = 0.3, tooltips = tooltipContent) { 
        x = "x"
        y = "y"
        fill = asDiscrete("n", orderBy = "..count..", order = -1)
        weight = "s"
        size = "..sum.."
    } +
    xlim(0.5 to 2.5) + ylim(0.5 to 2.5)

Mapping `fill` and `size` to the same variable:

In [13]:
letsPlot(data = mapOf("n" to listOf("a", "b", "c"), "s" to listOf(1, 2, 3))) + 
    geomPie(stat = Stat.identity) { fill = "n"; slice = "s"; size = "n" } +
    blankTheme

## 3. Explode

Use values to explode slices away from their center point, detaching it from the main pie.

In [14]:
val length = mapOf(
  "name" to listOf("20-50 km", "50-75 km", "10-20 km", "75-100 km", "3-5 km", "7-10 km", "5-7 km", ">100 km", "2-3 km"),
  "count" to listOf(1109, 696, 353, 192, 168, 86, 74, 65, 53),
  "explode" to listOf(0, 0, 0, 0.1, 0.1, 0.2, 0.3, 0.4, 0.6)
)

ggplot(length) + blankTheme +
    geomPie(stat = Stat.identity, stroke = 1, strokeColor = "black", size = 20) {
        fill = "name"; slice = "count"; explode = "explode"
    } +
    scaleFillGradient(low = "dark_blue", high = "light_green")


In [15]:
val calories = mapOf(
    "slice" to listOf(35, 25, 25, 15),
    "label" to listOf("Apples", "Bananas", "Cherries", "Dates"),
    "explode" to listOf(0.1, 0, 0, 0)
)

val p4 = ggplot(calories) +
    blankTheme +
    scaleFillBrewer(palette="Set1") +
    ggsize(w, h)

gggrid(plots = listOf(
    p4 + geomPie(stat = Stat.identity, size = 18) {
        fill = "label"; slice = "slice"; explode = "explode"
    } + theme().legendPositionNone(),
    p4 + geomPie(stat = Stat.identity, size = 18, hole = 0.8) {
        fill = "label"; slice = "slice"; explode = "explode"
    }),
       ncol = 2,
       cellWidth = w,
       cellHeight = h
)